In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import zipfile

# Unzip dataset
zip_file = "/content/train.zip"
extract_folder = "/content/trainimages"
csv_file = "/content/trainLabels.csv"

if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

with zipfile.ZipFile(zip_file, 'r') as archive:
    archive.extractall(extract_folder)

image_folder = "/content/trainimages/train"
df = pd.read_csv(csv_file)

# Load and preprocess image
def load_image(image_id, img_size=(28, 28)):
    img_path = os.path.join(image_folder, f"{image_id}.png")
    if os.path.exists(img_path):
        img = imread(img_path)
        img_resized = resize(img, img_size)
        img_gray = rgb2gray(img_resized)
        return img_gray
    else:
        print(f"Image {image_id}.png not found!")
        return None

X_data, y_data = [], []
for index, row in df.iterrows():
    img = load_image(row["id"])
    if img is not None:
        X_data.append(img)
        y_data.append(row["label"])

X_data = np.array(X_data).reshape(len(X_data), 28, 28, 1)
y_data = np.array(pd.factorize(y_data)[0])

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# CNN model architecture
def create_cnn():
    model = Sequential([
        Conv2D(64, (3,3), input_shape=(28, 28, 1), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(0.25),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(0.25),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    return model

# Train and evaluate with different optimizers
def train_with_optimizer(optimizer, name="Optimizer"):
    print(f"\n🔁 Training with {name}")
    
    model = create_cnn()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, verbose=0)

    # Evaluate
    score = model.evaluate(X_test, y_test, verbose=0)
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    
    print(f"📊 {name} Results:")
    print(f"Loss: {score[0]:.4f}, Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-score: {f1:.4f}")
    
    return {
        'name': name,
        'loss': score[0],
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1_score': f1
    }

# Run experiments
results = []
results.append(train_with_optimizer(Adam(), name="Adam"))
results.append(train_with_optimizer(SGD(momentum=0.9), name="SGD"))
results.append(train_with_optimizer(RMSprop(), name="RMSprop"))
results.append(train_with_optimizer(Adagrad(), name="Adagrad"))

# Results table
df_results = pd.DataFrame(results)
print("\n🔍 Optimizer Comparison Table:")
print(df_results)

# Plotting
df_results.set_index('name')[['accuracy', 'precision', 'recall', 'f1_score']].plot(kind='bar', figsize=(10,6), title="Optimizer Comparison")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.grid(True)
plt.show()
